# **🌟 LangChain + LangGraph Agents: Tools, Search, Multi-Step Reasoning & Chatbots**

This notebook introduces how to build **AI agents** using **LangChain** and **LangGraph**, combining LLMs with external tools, search APIs, and multi-step workflows.
You’ll start with simple agents and move into fully custom agent pipelines using LangGraph.

<br>

## 🔍 **What You Will Learn (Short & Friendly)**

* How to use LangChain’s `initialize_agent` with tools
* How to add external tools like Tavily Search
* How to build custom tools (date tool, currency exchange tool)
* How LangGraph creates multi-step agent pipelines
* How to build chatbots with and without memory
* How to use checkpointers like `MemorySaver` for persistent state

<br>

## 🧠 **Prerequisites**

* Basic Python
* Gemini API Key
* Tavily API Key (for search tool)
* Some familiarity with prompts and LLM calls

<br>

## 🚀 **Goal of This Notebook**

To help you build **real, working agents** — from basic single-tool agents to structured multi-step agents using LangGraph.

By the end, you'll know how agents connect LLM reasoning + external tools + stateful workflows.

<br>

## 📝 **Note**

This notebook documents my learning journey.
The implementations include the actual issues I faced and how I solved them.
They may not be production-ready but are designed to be practical and beginner-friendly.

#**LangChain + LangGraph Agent Demo**

## Environment setup

### **Install packages**
> Note: Dependency warnings may appear in Colab; proceed unless errors halt execution.

In [ ]:
!pip install -qU langchain-google-genai

In [ ]:
!pip install grandalf

## Configure API keys


In [ ]:
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
TAVILY_API_KEY = userdata.get('TAVILY_API_KEY')

## Initialize LLM
Model name, basic invocation, and sanity check.

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model='gemini-2.5-flash-lite',google_api_key=GOOGLE_API_KEY)
response = llm.invoke("What is Your knowledge Cutoff!")
print(response.content)

My knowledge cutoff is **June 2024**. Therefore, I cannot provide you with any information about events or developments that have occurred since that time.


In [ ]:
!pip install langchain langchain_community

## LangChain agent demo

In [ ]:
from langchain.agents import initialize_agent
from langchain_community.tools import TavilySearchResults

search = TavilySearchResults(tavily_api_key=TAVILY_API_KEY)
tools_list = [search]

agent = initialize_agent(tools = tools_list,llm=llm,verbose=True,agent="zero-shot-react-description")

agent.invoke("What’s the latest Telugu movie released this month?")

/tmp/ipython-input-3250165099.py:5: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  search = TavilySearchResults(tavily_api_key=TAVILY_API_KEY)
/tmp/ipython-input-3250165099.py:8: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to



> Entering new AgentExecutor chain...
I need to find the latest Telugu movie released in the current month. I will use the search tool to find this information.
Action: tavily_search_results_json
Action Input: latest Telugu movie released this month
Observation: [{'title': 'October 2025 Telugu Movies Release Date, Schedule & Calendar', 'url': 'https://www.filmibeat.com/telugu/movies/october-2025.html', 'content': "### Latest Releases\n\n Mrithyunjay  - 3 Oct 2025\n Chiru Odela Cinema  - 9 Oct 2025\n Sasivadane  - 10 Oct 2025\n Mahendragiri Varahi  - 11 Oct 2025\n Chennai Love Story  - 12 Oct 2025\n Sadhu  - 13 Oct 2025\n Don't Trouble The Trouble  - 14 Oct 2025\n Mithra Mandali  - 16 Oct 2025\n Garividi Lakshmi  - 17 Oct 2025\n NC 24  - 18 Oct 2025\n\n### Latest Box Office Collections\n\nNotifications\n\nClear Notifications\n\nNo New Notifications\n\nNotifications\n\nUse the toggle to switch on notifications [...] | Release Date | Movies Name | Cast |\n --- \n| 02 Oct 2025 | Kantara 

{'input': 'What’s the latest Telugu movie released this month?',
 'output': 'The latest Telugu movies released this month are "Devara" and "Meiyazhagan," both released on September 27, 2024.'}

## Langchain -> Custom tool: current date

Definition and example calls.

In [ ]:
response = llm.invoke("What is the current Date?")
print(response.content)

I do not have access to current date and time information. My knowledge cutoff is **June 2024**. Therefore, I cannot tell you the current date.


In [ ]:
import datetime
from langchain.agents import initialize_agent,tool

@tool
def get_current_date(text : str) -> str :
  """Return today's date, use this tool if you have any questions regarding the current date"""
  return datetime.datetime.now()

agent1 = initialize_agent(tools=[get_current_date],llm=llm,verbose=True,agent="zero-shot-react-description")

agent1.invoke("What is the current date?")



> Entering new AgentExecutor chain...
The user is asking for the current date. I have a tool `get_current_date` that can provide this information.
Action: get_current_date
Action Input: What is the current date?
Observation: 2025-10-10 05:10:36.340561
Thought:I have successfully retrieved the current date using the `get_current_date` tool. I can now answer the user's question.
Final Answer: The current date is 2025-10-10 05:10:36.340561.

> Finished chain.


{'input': 'What is the current date?',
 'output': 'The current date is 2025-10-10 05:10:36.340561.'}

In [ ]:
agent1.invoke("What is the yesterday date?")



> Entering new AgentExecutor chain...
The user is asking for yesterday's date. To determine this, I first need to know today's date. The `get_current_date` tool can provide this information.
Action: get_current_date
Action Input: today
Observation: 2025-10-10 05:10:37.256594
Thought:I have today's date, which is 2025-10-10. To find yesterday's date, I need to subtract one day from today's date. This can be done by manipulating the date string.
Final Answer: Yesterday's date was 2025-10-09.

> Finished chain.


{'input': 'What is the yesterday date?',
 'output': "Yesterday's date was 2025-10-09."}

## LangChain -> Custom tool: exchange rates

Inputs, outputs, timestamps, and sample queries.


📘 **Model Response Without Exchange Tool**

Before using the exchange-rate tool in the next cell, the example below shows how the model responds **without any external data/tool connection**. This means the answer may be **uncertain or inaccurate** 🤖💭.

⚠️ **Important:**
In this mode, the model is generating answers **purely from its internal knowledge**, which may be **uncertain, outdated, or completely inaccurate** 🤖💭.
This demonstration is provided to highlight the difference between an **uninformed model guess** and a **tool-assisted, reliable result**.


In [ ]:
response = llm.invoke("What is the current exchange rate from Indian Rupee (INR) to US Dollar (USD)?")
print(response.content)

I cannot provide you with the absolute "current" exchange rate in real-time because I am a language model and do not have access to live, constantly updating financial data. Exchange rates fluctuate by the minute.

**To get the most up-to-date INR to USD exchange rate, I recommend checking one of the following reliable sources:**

*   **Major Financial News Websites:**
    *   Bloomberg
    *   Reuters
    *   The Wall Street Journal
    *   The Financial Times
*   **Currency Converter Websites/Apps:**
    *   Google Search (simply type "INR to USD exchange rate")
    *   XE.com
    *   OANDA
    *   Your bank's website or app
*   **Financial Data Providers:**
    *   Forex.com
    *   Investing.com

When you search, you'll typically see a rate like:

**1 USD = X INR**

or

**1 INR = Y USD**

**As of my last knowledge update, the exchange rate has been hovering around:**

*   **1 US Dollar (USD) is approximately equal to 83 Indian Rupees (INR).**

**However, please do not rely on this 

🔧 **Tool-Assisted Exchange Rate Response**

In the cell below, the model uses the **exchange-rate tool** to fetch data from an external source.
This provides a **reliable and up-to-date** exchange rate, unlike the model-only response above.

In [ ]:
from langchain.agents import tool
import json
import requests

@tool
def get_exchange_rates(base_currency: str = "USD"):
    """
    Fetch latest exchange rates using the Exchange Rate API.

    Args:
        base_currency (str): The base currency code (e.g., 'USD', 'EUR').

    Returns:
        dict or None: JSON response containing exchange rates if successful, else None.
    """
    print("Debug step : ",base_currency," type of base_currency : ",type(base_currency))
    # Check if input is a JSON string
    try:
        # Attempt to parse JSON if possible
        parsed = json.loads(base_currency)
        if isinstance(parsed, dict) and "base_currency" in parsed:
            base_currency = parsed["base_currency"]
    except (json.JSONDecodeError, TypeError):
        # Not JSON, just use as-is
        pass

    base_currency = base_currency.upper()
    url = f"https://open.er-api.com/v6/latest/{base_currency}"

    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        if data.get("result") == "success":
            return data
        else:
            print("Failed to fetch data:", data.get("error-type", "Unknown error"))
            return None
    except requests.exceptions.RequestException as e:
        print(f"Error fetching exchange rates: {e}")
        return None


In [ ]:
agent = initialize_agent(
    tools=[get_exchange_rates],
    llm=llm,  # your LLM object
    agent="zero-shot-react-description",
    verbose=True
)
agent.invoke("What is the current exchange rate from Indian Rupee (INR) to US Dollar (USD)?")



> Entering new AgentExecutor chain...
I need to find the exchange rate from INR to USD. The `get_exchange_rates` tool can fetch exchange rates. I should set 'INR' as the base currency to get the rates from INR to other currencies, including USD.
Action: get_exchange_rates
Action Input: {"base_currency": "INR"}Debug step :  {"base_currency": "INR"}  type of base_currency :  <class 'str'>

Observation: {'result': 'success', 'provider': 'https://www.exchangerate-api.com', 'documentation': 'https://www.exchangerate-api.com/docs/free', 'terms_of_use': 'https://www.exchangerate-api.com/terms', 'time_last_update_unix': 1760054552, 'time_last_update_utc': 'Fri, 10 Oct 2025 00:02:32 +0000', 'time_next_update_unix': 1760141502, 'time_next_update_utc': 'Sat, 11 Oct 2025 00:11:42 +0000', 'time_eol_unix': 0, 'base_code': 'INR', 'rates': {'INR': 1, 'AED': 0.041328, 'AFN': 0.749942, 'ALL': 0.937434, 'AMD': 4.312799, 'ANG': 0.020144, 'AOA': 10.508763, 'ARS': 16.33719, 'AUD': 0.017127, 'AWG': 0.02014

{'input': 'What is the current exchange rate from Indian Rupee (INR) to US Dollar (USD)?',
 'output': 'The current exchange rate from Indian Rupee (INR) to US Dollar (USD) is 0.011253.'}

In [ ]:
agent.invoke("What is the current exchange rate from Indian Rupee (INR) to US Currency and japanese currency and china?")



> Entering new AgentExecutor chain...
The user is asking for the exchange rate of Indian Rupee (INR) to US Dollar, Japanese Yen, and Chinese Yuan. To get this information, I need to use the `get_exchange_rates` tool. The `get_exchange_rates` tool takes a `base_currency` as input. Since the user is asking for rates *from* INR, I should set the `base_currency` to 'INR'. The tool will then return rates of other currencies *against* INR. I will then need to extract the rates for USD, JPY, and CNY from the tool's output.

Action:
get_exchange_rates
Action Input:
{"base_currency": "INR"}Debug step :  {"base_currency": "INR"}  type of base_currency :  <class 'str'>

Observation: {'result': 'success', 'provider': 'https://www.exchangerate-api.com', 'documentation': 'https://www.exchangerate-api.com/docs/free', 'terms_of_use': 'https://www.exchangerate-api.com/terms', 'time_last_update_unix': 1760054552, 'time_last_update_utc': 'Fri, 10 Oct 2025 00:02:32 +0000', 'time_next_update_unix': 17601

{'input': 'What is the current exchange rate from Indian Rupee (INR) to US Currency and japanese currency and china?',
 'output': 'The current exchange rate from Indian Rupee (INR) to US Dollar is 1 INR = 0.011253 USD, to Japanese Yen is 1 INR = 1.722024 JPY, and to Chinese Yuan is 1 INR = 0.080251 CNY.'}

## LangGraph Agent Demo

This section demonstrates building a multi-step agent using LangGraph.

`langgraph` is installed to build stateful, multi-step agent workflows. `langchain-core` provides the fundamental components used in these workflows.

In [ ]:
!pip install langgraph

In [ ]:
!pip install langchain_core

In [ ]:
from langgraph.graph import MessageGraph,END
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_community.tools import TavilySearchResults

In [ ]:
from pprint import pprint
from datetime import datetime
current_date = datetime.now().strftime("%d %B %Y")  # e.g., "09 October 2025"



search = TavilySearchResults(tavily_api_key=TAVILY_API_KEY)

def planner_node(state):
  user_input = state[-1].content
  prompt = (f'''
You are a world-class research and planning assistant that builds **perfect online search queries** for any user question. Your task is to:
1. Determine if the user's query requires an online search to give an **accurate, up-to-date answer**.
2. If a search is needed, output **only the single most optimized search query** — nothing else.
3. Convert all **relative time references** to absolute dates:
   - "today" → "{current_date}"
   - "this week" → "week of {current_date}"
   - "this month" → "{current_date}"
   - "next month" → next calendar month, etc.
4. Correct typos in the query (e.g., "tday" → "today", "tis mnt" → "this month").
5. Be specific and unambiguous: include key attributes like type, language, or location if relevant.
6. Output should be **ready-to-use for TavilySearch**. **Do not include explanations, context, or commentary.**
7. **Always prioritize accuracy and temporal relevance.**

User query: "{user_input}"
''')

  response = llm.invoke(
      [
          SystemMessage(content=prompt),
          HumanMessage(content=user_input)
      ]
  )

  return state + [AIMessage(content=response.content)]

def search_node(state):
  query =state[-1].content
  search_results = search.invoke(query)
  print("Search result : \n")
  pprint(search_results)
  return state + [HumanMessage(content=str(search_results))]

def responder_node(state):
  user_question = state[0].content
  search_results = state[-1].content
  prompt = (f"""
You are a professional assistant that builds accurate, user-friendly responses from search results.
The user may ask about items, events, releases, or updates happening today, this month, or recently, and the input may contain typos (e.g., 'tday', 'tis mnt').
You are provided with search results that may include titles, names, dates, or release information.
Today's date is **{current_date}**. Your task is to:

1. Understand the user query, correcting any typos if necessary.
2. From the search results, identify the most recent item(s) or event(s) **up to today's date**.
4. Provide a clear, concise response including the name/title and date in a readable format for the user.
5. If no relevant items/events have occurred so far, clearly state that. Always prioritize accuracy based on today's date.
""")
  response = llm.invoke(
      [
          SystemMessage(content=prompt),
          HumanMessage(content=f"query by user : {user_question}"),
          AIMessage(content=f"search results from a tool or AI : {search_results}")
      ]
  )
  return state + [AIMessage(content=response.content)]

graph = MessageGraph()
graph.add_node("Planner",planner_node)
graph.add_node("Search",search_node)
graph.add_node("Responder",responder_node)

graph.set_entry_point("Planner")

graph.add_edge("Planner","Search")
graph.add_edge("Search","Responder")
graph.add_edge("Responder",END)

agent = graph.compile()

agent_response = agent.invoke([HumanMessage(content="What’s the latest Telugu movie released this month?")])
print(agent_response)

/tmp/ipython-input-536690329.py:67: LangGraphDeprecatedSinceV10: MessageGraph is deprecated in LangGraph v1.0.0, to be removed in v2.0.0. Please use StateGraph with a `messages` key instead. Deprecated in LangGraph V1.0 to be removed in V2.0.
  graph = MessageGraph()


Search result : 

[{'content': 'Movies | Music | Masti\n'
             'Movies | Music | Music\n'
             'Movies | Music | Music\n'
             '\n'
             '# October 2025 Telugu Movies\n'
             '\n'
             'October 2025 glimpsed successful Telugu Movies Idli Kottu, '
             'Kantara: Chapter-1, Ari, Erra Cheera, Sasivadane, Mithra '
             'Mandali, DUDE, Telusu Kada, K-RAMP, Mass Jathara.Here, Check out '
             'the deep-dyed rundown of October 2025 Telugu Movies & accessible '
             'OTT details.\n'
             '\n'
             '## October 2025 Movie Releases\n'
             '\n'
             '### Idli Kottu\n'
             '\n'
             '### Kantara: Chapter-1\n'
             '\n'
             '### Ari\n'
             '\n'
             '### Erra Cheera\n'
             '\n'
             '### Sasivadane\n'
             '\n'
             '### Mithra Mandali [...] ### DUDE\n'
             '\n'
             '### Telusu Kada\n'
  

In [ ]:
print(type(agent_response))
print(type(agent_response[0]))

<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>


In [ ]:
for message in agent_response:
  print(f"{message.type}    :   {message.content}")

human    :   What’s the latest Telugu movie released this month?
ai    :   latest Telugu movies released October 2025
human    :   [{'title': 'October 2025 Telugu Movies | OTT Updates - Ragalahari', 'url': 'https://www.ragalahari.com/movies/releases/october-2025.aspx', 'content': 'Movies | Music | Masti\nMovies | Music | Music\nMovies | Music | Music\n\n# October 2025 Telugu Movies\n\nOctober 2025 glimpsed successful Telugu Movies Idli Kottu, Kantara: Chapter-1, Ari, Erra Cheera, Sasivadane, Mithra Mandali, DUDE, Telusu Kada, K-RAMP, Mass Jathara.Here, Check out the deep-dyed rundown of October 2025 Telugu Movies & accessible OTT details.\n\n## October 2025 Movie Releases\n\n### Idli Kottu\n\n### Kantara: Chapter-1\n\n### Ari\n\n### Erra Cheera\n\n### Sasivadane\n\n### Mithra Mandali [...] ### DUDE\n\n### Telusu Kada\n\n### K-RAMP\n\n### Mass Jathara\n\n#### Latest News\n\nRead More News\n\n#### Latest Reviews\n\nRead More Reviews\n\n#### Latest Updates\n\nView More Updates\n\nCopyrigh

In [ ]:
agent.get_graph().print_ascii()

+-----------+  
| __start__ |  
+-----------+  
      *        
      *        
      *        
 +---------+   
 | Planner |   
 +---------+   
      *        
      *        
      *        
  +--------+   
  | Search |   
  +--------+   
      *        
      *        
      *        
+-----------+  
| Responder |  
+-----------+  
      *        
      *        
      *        
 +---------+   
 | __end__ |   
 +---------+   


## LangGraph writer–reviewer loop

In [ ]:
from langgraph.graph import MessageGraph, END
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate

### Prompts

Writer and Reviewer criteria and structure.

In [ ]:
writer_prompt = ChatPromptTemplate.from_template(
    """
      You are LinkedInWriterPro, an expert social-media content creator trained to write high-impact LinkedIn posts
      that build credibility, engage readers, and spark conversation.

      💡 Topic: {topic}

      Writing Guidelines:

      Tone: professional yet conversational — insightful, warm, and human.

      Structure: hook → insight/story → key takeaway → closing question or CTA.

      Language: clear, simple, and emotionally intelligent — avoid buzzwords or clichés.

      Formatting:

      Use short paragraphs and whitespace for readability.

      Bold or emoji highlights are optional — use them naturally, not decoratively.

      Length: ~150–250 words (unless user specifies otherwise).

      Hashtags: include 3–5 relevant hashtags at the end.

      Output format (always):

      💡 Topic: <user topic>

      ✍️ LinkedIn Post:
      <crafted post>

      #Hashtag1 #Hashtag2 #Hashtag3
    """
)

writer_chain = writer_prompt | llm

reviewer_prompt = ChatPromptTemplate.from_template(
    """
        You are LinkedInReviewPro, a professional LinkedIn strategist and content reviewer.
        Your task is to analyze LinkedIn posts and give specific, actionable suggestions to increase:

        👀 Reach (visibility, algorithm friendliness)

        💬 Engagement (likes, comments, saves, shares)

        👥 Followers (brand growth and personal authority)

        Your Review Style:

        Professional, concise, and insight-driven.

        No fluff — focus on what improves results and why.

        Respect the author’s voice — suggest enhancements, not rewrites.

        Evaluation Criteria:

        Hook Strength – Does the first 1–2 lines stop scrolling?

        Clarity & Flow – Easy to read? Logical transitions?

        Authenticity & Tone – Feels genuine and human?

        Engagement Triggers – Includes questions, relatable moments, or calls-to-action?

        Formatting – Good use of spacing, emojis, and emphasis for readability?

        Hashtags & Keywords – Relevant, limited (3–5), and algorithm-friendly?

        📄 Post to Review: {post}


        Output Format:

        🔍 Review Summary:
        <2–3 lines overview of the post’s current effectiveness>

        💡 Suggestions to Improve:
        1. <specific actionable suggestion to increase reach>
        2. <specific actionable suggestion to boost engagement>
        3. <specific actionable suggestion to grow followers>

        ✅ Optional Enhancements:
        <optional ideas for tone, structure, or hashtags>
    """
)

reviewer_chain = reviewer_prompt | llm

### Graph construction

Nodes, edges, compile, and optional ASCII graph.

In [ ]:
graph = MessageGraph()

def writer_node(state):
  final_input=""
  if len(state) == 1:
        # 🔹 First iteration — generate a new post from topic
        topic = state[-1].content
        final_input = topic
  else:
    # 🔹 Later iterations — refine old post using reviewer feedback
    old_post = state[-2].content
    review_feedback = state[-1].content

    final_input = f"""
        Refine and improve the following LinkedIn post based on the review feedback.
        Maintain tone and personality, but enhance engagement, reach, and clarity.

        --- Previous Post ---
        {old_post}

        --- Review Feedback ---
        {review_feedback}

        Output a polished, final version.
    """
  response = writer_chain.invoke({"topic": final_input})
  return state + [AIMessage(content=response.content)]

def reviewer_node(state):
  post = state[-1].content
  response = reviewer_chain.invoke({"post":post})
  return state + [AIMessage(content=response.content)]

graph.add_node("Writer",writer_node)
graph.add_node("Reviewer",reviewer_node)

graph.set_entry_point("Writer")

def condition_check(state):
  if len(state) >= 6 :
    return END
  else:
    return "Reviewer"

graph.add_conditional_edges("Writer", condition_check)
graph.add_edge("Reviewer","Writer")

app = graph.compile()

/tmp/ipython-input-1674853098.py:1: LangGraphDeprecatedSinceV10: MessageGraph is deprecated in LangGraph v1.0.0, to be removed in v2.0.0. Please use StateGraph with a `messages` key instead. Deprecated in LangGraph V1.0 to be removed in V2.0.
  graph = MessageGraph()


### Sample run

Example input and refined outputs.

In [ ]:
response = app.invoke(HumanMessage(content="What is Quantum AI?"))
for message in response:
  print(f"{message.type}    :   {message.content}")

human    :   What is Quantum AI?
ai    :   💡 Topic: What is Quantum AI?

✍️ LinkedIn Post:
Ever heard the term "Quantum AI" and wondered what it actually means? It's not just science fiction anymore – it's a fascinating frontier where two of the most revolutionary fields in technology are merging.

Think of it this way: Traditional AI excels at processing vast amounts of data. Quantum computing, on the other hand, harnesses the mind-bending principles of quantum mechanics to perform calculations that are impossible for even the most powerful supercomputers today.

When you combine these, you get Quantum AI. This powerful synergy promises to unlock solutions to problems we can only dream of tackling now. Imagine accelerating drug discovery, optimizing complex logistical networks, or creating truly intelligent materials – all at speeds and scales previously unimaginable.

It’s about building AI that can learn and solve problems in entirely new ways, leveraging the unique properties of qu

In [ ]:
app.get_graph().print_ascii()

  +-----------+    
  | __start__ |    
  +-----------+    
        *          
        *          
        *          
+---------------+  
| IncrementNode |  
+---------------+  
        *          
        *          
        *          
   +---------+     
   | __end__ |     
   +---------+     


> **Note:** The ASCII representation of the LangGraph only shows static edges. Conditional edges, while present in the graph's logic, are not explicitly shown in this visualization.



## StateGraph Demonstration

In [ ]:
from langgraph.graph import StateGraph,END
from typing import TypedDict


class MyState(TypedDict):
  count:int

def increse_count(state:MyState):
  state["count"] +=1
  return state


graph = StateGraph(MyState)

graph.add_node("IncrementNode",increse_count)

graph.set_entry_point("IncrementNode")

def count_checker(state:MyState):
  if state["count"] >= 5:
    return END
  else:
    return "IncrementNode"

graph.add_conditional_edges("IncrementNode",count_checker)

app = graph.compile()

response = app.invoke(MyState(count=0))
print(response)

{'count': 5}


In [ ]:
app.get_graph().print_ascii()

  +-----------+    
  | __start__ |    
  +-----------+    
        *          
        *          
        *          
+---------------+  
| IncrementNode |  
+---------------+  
        *          
        *          
        *          
   +---------+     
   | __end__ |     
   +---------+     


##ChatBot without memory

Here, `Annotated` and `add_messages` are used with the `messages` key in the `MyState` TypedDict. This special annotation tells LangGraph to automatically merge any new messages added to this list with the existing messages in the state, rather than replacing the entire list. This is crucial for building chatbots and agents that need to maintain a conversation history and reduces the need for manual appending of responses.

In [ ]:
from langgraph.graph import StateGraph, END, add_messages
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from typing import TypedDict,Annotated

In [ ]:
class MyState(TypedDict):
  messages:Annotated[list,add_messages]


chatbot_graph = StateGraph(MyState)

def chat_node(state:MyState):
  response = llm.invoke(state["messages"])
  state["messages"] = [response]
  return state


chatbot_graph.add_node("ChatNode",chat_node)

chatbot_graph.set_entry_point("ChatNode")

chatbot_graph.add_edge("ChatNode",END)

chatbot_app = chatbot_graph.compile()

response = chatbot_app.invoke({"messages":[HumanMessage(content="Hello i am Viswa Kambham!!")]})
print(response)


{'messages': [HumanMessage(content='Hello i am Viswa Kambham!!', additional_kwargs={}, response_metadata={}, id='2d1361c1-0360-463e-a728-3d93d021fb2c'), AIMessage(content="Hello Viswa Kambham! It's nice to meet you. How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': []}, id='run--813266f6-3308-42d0-9f7a-d217e50a561d-0', usage_metadata={'input_tokens': 9, 'output_tokens': 21, 'total_tokens': 30, 'input_token_details': {'cache_read': 0}})]}


In [ ]:
response = chatbot_app.invoke({"messages":[HumanMessage(content="what is my name?!")]})
print(response)

{'messages': [HumanMessage(content='what is my name?!', additional_kwargs={}, response_metadata={}, id='c3e7b367-f295-43d7-bec8-0537440e1b62'), AIMessage(content='I do not have access to your personal information, including your name. I am a large language model, an AI, and I do not collect or store any user data. Therefore, I cannot know your name.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': []}, id='run--12728967-77dc-4622-bdb5-eae6b1634e6b-0', usage_metadata={'input_tokens': 6, 'output_tokens': 43, 'total_tokens': 49, 'input_token_details': {'cache_read': 0}})]}


## ChatBot with Memory

### **LangGraph Checkpoint Notes**

**1️⃣ Checkpointer**

* **Definition:** An interface in LangGraph for saving and restoring graph state (conversation memory, variables, etc.).
* **Purpose:** Allows the graph to continue a conversation across multiple `invoke` calls.

**2️⃣ Types / Implementations**

* **MemorySaver:** Stores state **in RAM** (temporary, lost on restart).
* **SqliteSaver:** Stores state **on disk** (persistent across restarts).
* **RedisSaver / Custom:** Stores state in external database or service (shared across servers).


**Example:**

```python
memory = MemorySaver()
chatbot_app = chatbot_graph.compile(checkpointer=memory)
```

In [ ]:
from langgraph.graph import StateGraph, END, add_messages
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langgraph.checkpoint.memory import MemorySaver
from typing import TypedDict,Annotated

In [ ]:
class MyState(TypedDict):
  messages:Annotated[list,add_messages]


chatbot_graph = StateGraph(MyState)

memory =MemorySaver()

configuration = {
    "configurable" : {
        "thread_id":1
    }
}

def chat_node(state:MyState):
  response = llm.invoke(state["messages"])
  state["messages"] = [response]
  return state


chatbot_graph.add_node("ChatNode",chat_node)

chatbot_graph.set_entry_point("ChatNode")

chatbot_graph.add_edge("ChatNode",END)

chatbot_app = chatbot_graph.compile(checkpointer=memory)

response = chatbot_app.invoke({"messages":[HumanMessage(content="Hello i am Viswa Kambham!!")]},config=configuration)
print(response)


{'messages': [HumanMessage(content='Hello i am Viswa Kambham!!', additional_kwargs={}, response_metadata={}, id='b954e21b-6c6f-47f0-9a28-d54f4b198695'), AIMessage(content="Hello Viswa Kambham! It's nice to meet you. How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': []}, id='run--1c6bdc58-536f-4037-a29a-c821adda803e-0', usage_metadata={'input_tokens': 9, 'output_tokens': 21, 'total_tokens': 30, 'input_token_details': {'cache_read': 0}})]}


In [ ]:
response = chatbot_app.invoke({"messages":[HumanMessage(content="what is my name?!")]},config=configuration)
print(response)

{'messages': [HumanMessage(content='Hello i am Viswa Kambham!!', additional_kwargs={}, response_metadata={}, id='b954e21b-6c6f-47f0-9a28-d54f4b198695'), AIMessage(content="Hello Viswa Kambham! It's nice to meet you. How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': []}, id='run--1c6bdc58-536f-4037-a29a-c821adda803e-0', usage_metadata={'input_tokens': 9, 'output_tokens': 21, 'total_tokens': 30, 'input_token_details': {'cache_read': 0}}), HumanMessage(content='what is my name?!', additional_kwargs={}, response_metadata={}, id='602aabe2-6867-452e-8ebf-f8e96a0add92'), AIMessage(content='Your name is **Viswa Kambham**. You just told me! 😊', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': []}, id=

# ✅ **Summary & Next Steps**

In this notebook, you explored how to build **agents** using LangChain and LangGraph.
Here’s what you completed:

### 🔹 **You Have Done:**

* Installed and configured LangChain + LangGraph
* Used Tavily Search as a tool inside LangChain agents
* Built a **zero-shot-react-description** agent
* Created custom tools, including:

  * `get_current_date`
  * `get_exchange_rates` using a real API
* Ran multi-step tool invocations inside LangChain
* Built a **multi-step agent workflow** using LangGraph nodes:

  * Planner → Search → Responder
* Visualized the LangGraph structure with ASCII
* Used LangGraph’s `StateGraph` for:

  * Counter example
  * Writer–Reviewer loop
* Built a **chatbot without memory**
* Added **memory** using Checkpointer (`MemorySaver`)
* Maintained conversation context using `Annotated` + `add_messages`

<br>

## 🧱 **What This Notebook Gives You**

You now understand how agents actually work:

* LLM → decides an action
* Tool → returns result
* Agent → reasons again
* Repeat → until final answer

You also learned how LangGraph structures multi-step flows using graph nodes and state transitions — the foundation for advanced AI systems.

<br>

## 🔮 **This prepares you for more advanced agent notebooks that involve:**

* Building real end-to-end agent applications (like the Career Agent)
* Multi-node workflows using LangGraph’s StateGraph
* Resume parsing and structured extraction using output parsers
* Integrating external APIs such as Tavily Search
* Multi-step agent pipelines that chain reasoning + tools + generation



<br>

💬 **Tip:**
Try adding your own tools, like weather, stock info, or custom APIs.
Agents become powerful when they can act, not just answer.

**Keep experimenting — this is where agents start becoming REAL. 🚀**